In [ ]:
!pip install scikit-learn

In [1]:
import numpy as np
import tensorflow as tf
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split

# Load dataset
digits = load_digits()
X = digits.images / 16.0
y = digits.target
X = X.reshape((-1, 8, 8, 1))
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Build tiny CNN
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(4, (3,3), activation='relu', input_shape=(8,8,1)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, activation='softmax')
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

# Representative dataset for int8 quantization
def representative_dataset():
    for i in range(len(X_train)):
        yield [X_train[i:i+1].astype(np.float32)]

# Convert to fully int8
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8
tflite_model = converter.convert()

# Save fully int8 TFLite model
with open("digit_model_int8.tflite", "wb") as f:
    f.write(tflite_model)


Epoch 1/10
45/45 [==============================] - 7s 15ms/step - loss: 2.2125 - accuracy: 0.2143 - val_loss: 1.9976 - val_accuracy: 0.4167
Epoch 2/10
45/45 [==============================] - 0s 4ms/step - loss: 1.7709 - accuracy: 0.6256 - val_loss: 1.4940 - val_accuracy: 0.8167
Epoch 3/10
45/45 [==============================] - 0s 4ms/step - loss: 1.2606 - accuracy: 0.8399 - val_loss: 1.0067 - val_accuracy: 0.9000
Epoch 4/10
45/45 [==============================] - 0s 3ms/step - loss: 0.8511 - accuracy: 0.8900 - val_loss: 0.6955 - val_accuracy: 0.9139
Epoch 5/10
45/45 [==============================] - 0s 4ms/step - loss: 0.6049 - accuracy: 0.9102 - val_loss: 0.5099 - val_accuracy: 0.9361
Epoch 6/10
45/45 [==============================] - 0s 4ms/step - loss: 0.4657 - accuracy: 0.9179 - val_loss: 0.4122 - val_accuracy: 0.9333
Epoch 7/10
45/45 [==============================] - 0s 4ms/step - loss: 0.3828 - accuracy: 0.9235 - val_loss: 0.3504 - val_accuracy: 0.9417
Epoch 8/10
45/45 [=

INFO:tensorflow:Assets written to: C:\Users\kukil\AppData\Local\Temp\tmpepbhw1d_\assets


INFO:tensorflow:Assets written to: C:\Users\kukil\AppData\Local\Temp\tmpepbhw1d_\assets
C:\Users\kukil\.conda\envs\tinyML\lib\site-packages\tensorflow\lite\python\convert.py:766: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "
